In [1]:
# Configuration
dbname = 'test'

In [2]:
from tqdm import tqdm_notebook as tqdm
from pymongo import MongoClient
client = MongoClient()

In [9]:
client.drop_database(dbname)
db = client[dbname]

In [4]:
from anki import Collection
col = Collection('testdata/collection.anki2', log=True)

In [10]:
for k, v in col.models.models.items():
    model = {
        '_id': v['id'],
        'name': v['name'],
        'fields': [
            {
                'name': fld['name'],
                'sticky': fld['sticky']
            } for fld in v['flds']
        ],
        'modTime': v['mod'],
        'templates': [
            {
                'frontFormat': tmpl['qfmt'],
                'answerFormat': tmpl['afmt'],
                'name': tmpl['name']
            } for tmpl in v['tmpls']
        ],
        'modelType': {
            0: 'basic',
            1: 'cloze',
        }[v['type']]
    }
    db.models.insert_one(model)

In [17]:
# Move revlog

REVLOG_LEARN = 0
REVLOG_REVIEW = 1
REVLOG_RELEARN = 2
REVLOG_CRAM = 3

revEntryList = []
for _id, cid, usn, ease, ivl, lastIvl, factor, time, _type in col.db.all("select * from revlog order by id desc"):
    revEntry = {
        'createdAt': _id / 1000,
        'card': cid,
        'reviewType': {
            0: REVLOG_LEARN,
            1: REVLOG_REVIEW,
            2: REVLOG_RELEARN,
            3: REVLOG_CRAM
        }[_type],
        'reviewTime': time / 1000,
        'response': ease
    }
    revEntryList.append(revEntry)
    
db.revlog.insert_many(revEntryList)

In [26]:
rtl = [x['reviewType'] for x in revEntryList]
rtl.count(1)

40500

In [12]:
SCHED_NORMAL = 0
SCHED_SUSPENDED = 1
SCHED_BURIED = 2

for nid in tqdm(col.findNotes("*")):
    note = col.getNote(nid)
    noteEntry = {
        '_id': note.id,
        'model': note.mid,
        'fields': note.fields,
        'tags': note.tags,
        'createdAt': note.id // 1000,
        'updatedAt': note.mod,
        'cards': []
    }
        
    for card in note.cards():
        db.cards.insert_one({
            '_id': card.id,
            'note': card.nid,
            'deck': card.did,
            'noteCardIndex': card.mod,
            'schedState': {
                -3: SCHED_BURIED,
                -2: SCHED_BURIED,
                -1: SCHED_SUSPENDED,
                0: SCHED_NORMAL,
                1: SCHED_NORMAL,
                2: SCHED_NORMAL,
                3: SCHED_NORMAL,
            }[card.queue]
        })
        noteEntry['cards'].append(card.id)
        
    db.notes.insert_one(noteEntry)


HBox(children=(IntProgress(value=0, max=14979), HTML(value='')))